In [2]:
from numba import njit
import numba as nb
import numpy as np
import wpimath.geometry as wpi
import scipy

FAST_MATH = True


@njit(
    nb.types.Array(nb.float64, 2, "C")(nb.types.Array(nb.float64, 1, "C")),
    parallel=True,
    fastmath=FAST_MATH,
)
def euler_to_matrix(euler):
    # assert euler.shape == (3,)

    sin_alpha, sin_beta, sin_gamma = np.sin(euler)
    cos_alpha, cos_beta, cos_gamma = np.cos(euler)

    return np.asarray(
        [
            [
                cos_beta * cos_gamma,
                sin_alpha * sin_beta * cos_gamma - sin_gamma * cos_alpha,
                sin_beta * cos_alpha * cos_gamma + sin_alpha * sin_gamma,
            ],
            [
                sin_gamma * cos_beta,
                sin_alpha * sin_beta * sin_gamma + cos_alpha * cos_gamma,
                sin_beta * sin_gamma * cos_alpha - sin_alpha * cos_gamma,
            ],
            [-sin_beta, sin_alpha * cos_beta, cos_alpha * cos_beta],
        ],
    )


@njit(
    nb.types.Array(nb.float64, 2, "C")(nb.types.Array(nb.float64, 1, "C")),
    parallel=True,
    fastmath=FAST_MATH,
)
def quaterion_to_matrix(q):
    # assert q.shape == (4,)

    w, x, y, z = q

    rotation_matrix = np.asarray(
        [
            [1 - 2 * (y * y + z * z), 2 * (x * y - w * z), 2 * (x * z + w * y)],
            [2 * (x * y + w * z), 1 - 2 * (x * x + z * z), 2 * (y * z - w * x)],
            [2 * (x * z - w * y), 2 * (y * z + w * x), 1 - 2 * (x * x + y * y)],
        ],
    )

    return rotation_matrix


# ROT MUST BE A 3x3 ROTATION MATRIX
@njit(
    nb.types.Array(nb.float64, 2, "C")(
        nb.types.Array(nb.float64, 1, "C"), nb.types.Array(nb.float64, 2, "C")
    ),
    parallel=True,
    fastmath=FAST_MATH,
)
def make_transform(trans, rot):
    # assert trans.shape == (3, 1) and rot.shape == (3, 3)

     return np.concatenate(
        (np.concatenate((rot, np.atleast_2d(trans).T), axis=1), np.asarray([[0, 0, 0, 1]])), axis=0
    )


@njit(
    nb.types.Array(nb.float64, 2, "C")(nb.types.Array(nb.float64, 1, "C")),
    parallel=True,
    fastmath=FAST_MATH,
)
def make_rotation_transform(euler):
    # assert euler.shape == (3,)

    return make_transform(np.zeros(3), euler_to_matrix(euler))


@njit(
    nb.types.Array(nb.float64, 2, "C")(nb.types.Array(nb.float64, 1, "C")),
    parallel=True,
    fastmath=FAST_MATH,
)
def make_translation_transform(translation):
    # assert translation.shape == (3,)

    return make_transform(translation, np.eye(3, 3))


@njit(
    nb.types.Array(nb.float64, 2, "A")(nb.types.Array(nb.float64, 2, "C")),
    parallel=True,
    fastmath=FAST_MATH,
)
def get_rotation(transform):
    # assert transform.shape == (4, 4)

    return transform[:3, :3]


@njit(
    nb.types.Array(nb.float64, 1, "A")(nb.types.Array(nb.float64, 2, "C")),
    parallel=True,
    fastmath=FAST_MATH,
)
def get_translation(transform):
    # assert transform.shape == (4, 4)

    return transform[:3, 3]


@njit(
    nb.types.Array(nb.float64, 1, "C")(nb.types.Array(nb.float64, 2, "C")),
    parallel=True,
    fastmath=FAST_MATH,
)
def matrix_to_euler(rotation_matrix):
    # assert rotation_matrix.shape == (3, 3)

    beta = -np.arcsin(rotation_matrix[2, 0])
    cos_beta = np.cos(beta)
    alpha = np.arctan2(
        rotation_matrix[2, 1] / cos_beta,
        rotation_matrix[2, 2] / cos_beta,
    )
    gamma = np.arctan2(
        rotation_matrix[1, 0] / cos_beta,
        rotation_matrix[0, 0] / cos_beta,
    )
    return np.asarray((alpha, beta, gamma))


# make_transform(
#     np.zeros((3, 1)), quaterion_to_matrix(np.asarray([1, 0, 0, 0]))
# ) @ make_transform(np.zeros((3, 1)), quaterion_to_matrix(np.asarray([1, 0, 0, 0])))

/home/dshen/.local/lib/python3.11/site-packages/numba/core/typed_passes.py:336: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../../tmp/ipykernel_790066/4038367442.py", line 38:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,
/home/dshen/.local/lib/python3.11/site-packages/numba/core/typed_passes.py:336: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../../tmp/ipykernel_790066/4038367442.py", line 60:
<source missing, REPL/exec in use?>

  warnings.warn(errors

In [3]:
def wpi_stuff():
    t = wpi.Transform3d(
        wpi.Translation3d(1.1, 2.1, 3.1), wpi.Rotation3d(wpi.Quaternion(0.7, 0.7, 0, 0))
    )
    t2 = wpi.Transform3d(
        wpi.Translation3d(3.1, 2.1, 1.1), wpi.Rotation3d(wpi.Quaternion(0.7, 0.7, 0, 0))
    )

    return t + t2


@njit(nb.types.Array(nb.float64, 2, "A")(), parallel=True, fastmath=FAST_MATH)
def np_stuff():
    t = make_transform(
        np.asarray([1.1, 2.1, 3.1]),
        quaterion_to_matrix(np.asarray((0.7, 0.7, 0, 0))),
    )

    t2 = make_transform(
        np.asarray([3.1, 2.1, 1.1]),
        quaterion_to_matrix(np.asarray((0.7, 0.7, 0, 0))),
    )
    return t @ t2

wpi_stuff()
np_stuff()

/home/dshen/.local/lib/python3.11/site-packages/numba/core/typed_passes.py:336: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../../tmp/ipykernel_790066/56540803.py", line 12:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


array([[ 1.    ,  0.    ,  0.    ,  4.2   ],
       [ 0.    , -0.96  , -0.0392,  1.064 ],
       [ 0.    ,  0.0392, -0.96  ,  5.18  ],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [4]:
%timeit -n 1000000 -r 5 np_stuff()
%timeit -n 100000 -r 5 wpi_stuff()

1.81 μs ± 30.9 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)
8.52 μs ± 48 ns per loop (mean ± std. dev. of 5 runs, 100,000 loops each)


In [6]:
# %timeit -n 10000 -r 5  x = make_rotation_transform(np.asarray([1, 0, 0], dtype=np.float64))
# %timeit -n 10000 -r 5  x = make_rotation_transform_old(np.asarray([1, 0, 0], dtype=np.float64))

In [21]:
t = wpi.Transform3d(
        wpi.Translation3d(1.1, 2.1, 3.1), wpi.Rotation3d(wpi.Quaternion(0.7, 0.7, 0, 0))
    )
t2 = wpi.Transform3d(
    wpi.Translation3d(3.1, 2.1, 1.1), wpi.Rotation3d(wpi.Quaternion(0.7, 0.7, 0, 0))
)

%timeit -n 1000000 -r 5 x = (t + t2)


t = make_transform(
        np.asarray([1.1, 2.1, 3.1]),
        quaterion_to_matrix(np.asarray((0.7, 0.7, 0, 0))),
    )

t2 = make_transform(
    np.asarray([3.1, 2.1, 1.1]),
    quaterion_to_matrix(np.asarray((0.7, 0.7, 0, 0))),
)

@njit
def test():
    return t @ t2
%timeit -n 1000000 -r 5 test()


979 ns ± 26.7 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)
710 ns ± 45.2 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)


In [1]:
import numpy as np

x = np.asarray([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23 , 24, 25, 26])
y = np.asarray([1, 2, 3, 4, 5])

%timeit np.i

AttributeError: module 'numpy' has no attribute 'i'